## Problem description
Forex is the market for trading currency pairs, it is similar to the stock market, but only for a currency pair. In order to trade in the market a chart which contains the historic price and a set of technical indicators (features) is used by the trader to predict the future price. The type of the technical indicators used and the length of the inspection window is up to the trader himself and based on his previous experience. **The end goal is to build a machine learning model to predict the market movements.**
![Forex Chart example](pics\chart_example.png)

## Target of this project 
- benchmark performance of **optimal** neural nets using Autodesigner
- test and compare performance for different sets of features (technical indicators) and time-windows
- come up with a proposal for the set of features to be used in next stage of modeling
- have a baseline performance which is the minimum acceptance criteria moving forward

In [1]:
# code imports
import pandas as pd
from forex.data.utils import fetch_data
from forex.processing.utils import data_construct
from matplotlib import pyplot as plt # (%matplotlib notebook) in case interactive graphs are needed 

Load the data for EUR/USD pair with the following configurations:
- granularity = 5 minutes
- count = 1000 candles

[fetch_data](.\forex\data\utils.py)

In [2]:
data = fetch_data(granularity='M5', count=1000, instrument='EUR_USD', return_list=False)

In [3]:
data.head()

,time,completed,volume,o,h,l,c
0,2019-11-05T10:15:00.000000000Z,True,88,1.11309,1.11309,1.11262,1.11292
1,2019-11-05T10:20:00.000000000Z,True,64,1.11290,1.11301,1.11263,1.11263
2,2019-11-05T10:25:00.000000000Z,True,67,1.11266,1.11288,1.11264,1.11282
3,2019-11-05T10:30:00.000000000Z,True,41,1.11279,1.11288,1.11276,1.11284
4,2019-11-05T10:35:00.000000000Z,True,54,1.11286,1.11312,1.11279,1.11308


Construct the train and targets matrices by
- Framing the data into **predict window** and **prediction window** and then keep sliding over the data
- Construct features for each window in a way to overcome the **dimension hopping** problem
- **Dimensionality reduction** into the most meaningful features, hence deacreasing the data points requirements

![Window chart example](pics\window.png)
[data_construct](.\forex\processing\utils.py)

In [18]:
# construct supervised learning data
output = data_construct(DataFrame=data, lookUp=10, predictionWindow=5, pairName='EUR/USD')
f_map = output['mappingDict']
raw_inputs = output['rawInputs']
x = output['inputFeatures']
y = output['targets']

print('Shape of features matrix: {} examples and {} features'.format(x.shape[0], x.shape[1]))
print('Shape of targets matrix: {} examples and {} target'.format(y.shape[0], y.shape[1]))

Shape of features matrix: 953 examples and 62 features
Shape of targets matrix: 953 examples and 1 target


Construct the four competing datasets for training aand evaluation
- dataset one: sma features
- dataset two: ema features
- dataset three: band features
- dataset four: macd features

In [19]:
# construct sma features
x_sma = x[:, [f_map['sma_latest_crossover'],
              f_map['sma_location_of_latest_crossover'],
              f_map['sma_latest_crossover_angle'],
              f_map['sma_dominant_type_fast_slow']]]
print('Shape of x_sma matrix: {} examples and {} features'.format(x_sma.shape[0], x_sma.shape[1]))

# construct ema features
x_ema = x[:, [f_map['ema_latest_crossover'],
              f_map['ema_location_of_latest_crossover'],
              f_map['ema_latest_crossover_angle'],
              f_map['ema_dominant_type_fast_slow']]]
print('Shape of x_ema matrix: {} examples and {} features'.format(x_ema.shape[0], x_ema.shape[1]))

# construct band features
x_band = x[:, [f_map['deltaBands_mean'],
               f_map['deltaBands_std'],
               f_map['deltaBands_maximum_mean'],
               f_map['deltaBands_maximum_location'],
               f_map['deltaBands_minimum_mean'],
               f_map['deltaBands_minimum_location']]]
print('Shape of x_band matrix: {} examples and {} features'.format(x_band.shape[0], x_band.shape[1]))

# construct macd features
x_macd = x[:, [f_map['macd_latest_crossover'],
               f_map['macd_location_of_latest_crossover'],
               f_map['macd_latest_crossover_angle'],
               f_map['macd_dominant_type_fast_slow']]]
print('Shape of x_macd matrix: {} examples and {} features'.format(x_macd.shape[0], x_macd.shape[1]))

Shape of x_sma matrix: 953 examples and 4 features
Shape of x_ema matrix: 953 examples and 4 features
Shape of x_band matrix: 953 examples and 6 features
Shape of x_macd matrix: 953 examples and 4 features
